## 1. Defining the Question(s)

### Central Goal  
Can we build and evaluate a machine learning model that accurately predict the outcome of a men's international football match (Home Win, Draw, or Away Win) using historical match data?

### Secondary Questions  
- What features matter most for predicting match outcomes? (e.g., recent form, historical dominance, home-field advantage)  
- Does model performance change across tournament types (World Cup vs. friendlies)?  
- Does the model show any bias, such as favoring top-ranked teams over underdogs?

### Import necessary Packages

In [10]:
import pandas as pd
import numpy as np

### Load the Datasets

In [11]:
results_df = pd.read_csv('data/results.csv')
goalscorers_df = pd.read_csv('data/goalscorers.csv')
formerNames_df = pd.read_csv('data/former_names.csv')
shootouts_df = pd.read_csv('data/shootouts.csv')
results_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


### Data Cleaning

In [13]:
# Will mostly use results_df
# Summary of the data
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48850 entries, 0 to 48849
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        48850 non-null  object 
 1   home_team   48850 non-null  object 
 2   away_team   48850 non-null  object 
 3   home_score  48849 non-null  float64
 4   away_score  48849 non-null  float64
 5   tournament  48850 non-null  object 
 6   city        48850 non-null  object 
 7   country     48850 non-null  object 
 8   neutral     48850 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 3.0+ MB


In [14]:
# Nummer of missing values in each column
results_df.isnull().sum()

date          0
home_team     0
away_team     0
home_score    1
away_score    1
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [ ]:
# Converting date into datatime